In [3]:

from rhs_slm import SLM
from rhs_tlm import TLM
from total_current import total_current_slm, total_current_tlm
from scipy.optimize import least_squares
from scipy.integrate import solve_ivp
from radauDAE import RadauDAE
from plot import plot_JV
from dimensionalize import dimensionalize_slm, dimensionalize_tlm

import numpy as np
import matplotlib.pyplot as plt

import parameters, grid, initial_conditions, mass_slm, jacobian_slm, mass_tlm, jacobian_tlm

#Common class objects used by both JV functions.
import time

params = parameters.Params()
grid = grid.Grid(params)
ic = initial_conditions.Initial_Conditions(params, grid)

#Common solver settings used by both JV functions.
tf = 1
method = RadauDAE
rtol = params.rtol
atol = params.atol

def JV(*model):
    if (’single-layer’ in model):
    
        #Create class objects, mass matrix, and jacobian under standard conditions.
        mass = mass_slm.Mass(params, grid)
        jac = jacobian_slm.Jac(params)
        mass = mass.M
        jac = jac.jac

        #Solution process for JV scan. Dense output is only used for the scanning solution procedures
        sol_init = ic.sol_init_slm
        dae_fun = lambda t, u: SLM(t, u, ’pbi’)
        
        print(’Eliminating␣transient␣behavior␣at␣built-in␣voltage.’)
        start_time = time.time()
        sol = solve_ivp(fun=dae_fun, t_span=(0.0, tf), y0=sol_init,rtol=rtol, atol=atol, jac_sparsity=jac, method=method, vectorized=False, dense_output=False, mass=mass)
        print(’Preconditioning␣device␣to␣Vi.’)
        
dae_fun = lambda t, u: SLM(t, u, ’precondition’)
sol_init = sol.y[:,-1]
sol = solve_ivp(fun=dae_fun, t_span=(0.0, 10*tf), y0=sol_init, rtol=rtol, atol=atol, jac_sparsity=jac, method=method, vectorized=False, dense_output=False, mass=mass)
sol_init = sol.y[:,-1]
tf_scan = params.tf_scan
dae_fun = lambda t, u: SLM(t, u, ’scan_1’)
print(’Beginning␣JV␣scan.’)

sol = solve_ivp(fun=dae_fun, t_span=(0.0, tf_scan), y0=sol_init, rtol=rtol, atol=atol, jac_sparsity=jac, method=method, vectorized=False, dense_output=True, mass=mass)
t_vector_1 = sol.t
u_matrix_1 = sol.y

#These conditions allow for the correct calculations for psi depending if the scan begins reverse or forward.

if params.Vi < params.Vf:
psi_1 = params.phi_precondition*np.ones(t_vector_1.size) +
params.psi_scan_rate*t_vector_1
elif params.Vi > params.Vf:
psi_1 = params.phi_precondition*np.ones(t_vector_1.size) -
params.psi_scan_rate*t_vector_1
J_reverse = total_current_slm(t_vector_1, u_matrix_1)[0]
sol_init = sol.y[:,-1]
dae_fun = lambda t, u: SLM(t, u, ’scan_2’)
print(’Scanning␣opposite␣direction.’)
sol = solve_ivp(fun=dae_fun, t_span=(0.0, tf_scan), y0=sol_init, rtol=rtol, atol=atol, jac_sparsity=jac, method=method, vectorized=False, dense_output=True, mass=mass)
print(’Scan␣Complete’)
t_vector_2 = sol.t
u_matrix_2 = sol.y

if params.Vi < params.Vf:
psi_2 = params.phi_f*np.ones(t_vector_2.size) - params.
psi_scan_rate*t_vector_2
elif params.Vi > params.Vf:
psi_2 = params.phi_f*np.ones(t_vector_2.size) + params.
psi_scan_rate*t_vector_2
J_forward = total_current_slm(t_vector_2, u_matrix_2)[0]

#Concatenate t_vectors and u_matrices into complete data structures for both forward and backward scans

t_non_dim = np.concatenate((t_vector_1, t_vector_1[-1] + t_vector_2), 0)
u_matrix = np.concatenate((u_matrix_1, u_matrix_2), 1)
psi = np.concatenate((psi_1, psi_2), 0)

#Dimensionalize results
dimensional_sol = dimensionalize_slm(grid.x, t_non_dim, u_matrix,
psi)
x = dimensional_sol[0]
t = dimensional_sol[1]
P = dimensional_sol[2]
phi = dimensional_sol[3]
n = dimensional_sol[4]
p = dimensional_sol[5]
J_total = dimensional_sol[6]
V_applied = dimensional_sol[7]

#Plot results
plot_JV(V_applied, J_total, ’dimensional’)
scan1_size = psi_1.size
scan2_size = psi_2.size

"""""
plt.plot(psi_1, J_reverse)
plt.plot(psi_2, J_forward)
plt.xlim([0,40])
plt.ylim([0,0.8])
"""

print("---␣%s␣seconds␣---" % (time.time() - start_time))
elif (’three-layer’ in model):
    
#Create class objects, mass matrix, and jacobian under standard conditions.
mass = mass_tlm.Mass(params, grid, mode=None)
jac = jacobian_tlm.Jac(params, mode=None)
mass = mass.M
jac = jac.jac

#Solution process for JV scan.- Dense output is only used for the scanning solution procedures.
sol_init = ic.sol_init_tlm
dae_fun = lambda t, u: TLM(t, u, ’pbi’, mode=None)
start_time = time.time()
print(’Eliminating␣transient␣behavior␣at␣built-in␣voltage.’)
sol = solve_ivp(fun=dae_fun, t_span=(0.0, tf), y0=sol_init, rtol=rtol, atol=atol, jac_sparsity=jac, method=method, vectorized=False, dense_output=False, mass=mass)
dae_fun = lambda t, u: TLM(t, u, ’precondition’, mode=None)
sol_init = sol.y[:,-1]

#Adjust class objects to increase ion mobility to precondition at a voltage other than Vbi. """
mass = mass_tlm.Mass(params, grid, mode=’precondition’)
jac = jacobian_tlm.Jac(params, mode=None)
mass = mass.M
jac = jac.jac
print(’Preconditioning␣device␣to␣Vi.’)
sol = solve_ivp(fun=dae_fun, t_span=(0.0, 10*tf), y0=sol_init,
rtol=rtol, atol=atol, jac_sparsity=jac,
method=method, vectorized=False, dense_output=
False,
mass=mass)
sol_precondition = sol.y[:,-1]
tf_precondition = sol.t[-1]

#Redefine jacobian to ensure conservation of ions for steady state solution.

jac = jacobian_tlm.Jac(params, mode=’precondition’)
jac = jac.jac
if params.Vi < params.Vbi:
sol_init = least_squares(lambda u: TLM(tf_precondition, u, ’
precondition’, mode=’precondition’), sol_precondition,
jac_sparsity=jac).x

else:
sol_init = sol.y[:,-1]

#Initial conditions for the JV scan are now calculated. Calculate non-dimensional scan time and reset class objects to normal conditions.

tf_scan = params.tf_scan
dae_fun = lambda t, u: TLM(t, u, ’scan_1’, mode=None)
mass = mass_tlm.Mass(params, grid, mode=None)
jac = jacobian_tlm.Jac(params, mode=None)
mass = mass.M
jac = jac.jac
print(’Beginning␣JV␣scan.’)
sol = solve_ivp(fun=dae_fun, t_span=(0.0, tf_scan), y0=sol_init, rtol=rtol, atol=atol, jac_sparsity=jac, method=method, vectorized=False, dense_output=True, mass=mass)
t_vector_1 = sol.t
u_matrix_1 = sol.y

#These conditions allow for the correct calculations for psi depending if the scan begins reverse or forward.
if params.Vi < params.Vf:
psi_1 = params.phi_precondition*np.ones(t_vector_1.size) +
params.psi_scan_rate*t_vector_1
elif params.Vi > params.Vf:
psi_1 = params.phi_precondition*np.ones(t_vector_1.size) -
params.psi_scan_rate*t_vector_1
sol_init = sol.y[:,-1]
dae_fun = lambda t, u: TLM(t, u, ’scan_2’, mode=None)
print(’Scanning␣opposite␣direction.’)
sol = solve_ivp(fun=dae_fun, t_span=(0.0, tf_scan), y0=sol_init, rtol=rtol, atol=atol, jac_sparsity=jac, method=method, vectorized=False, dense_output=True, mass=mass)
print(’Scan␣Complete’)
print("---␣%s␣seconds␣---" % (time.time() - start_time))

t_vector_2 = sol.t
u_matrix_2 = sol.y
if params.Vi < params.Vf:
psi_2 = params.phi_f*np.ones(t_vector_2.size) - params.
psi_scan_rate*t_vector_2
elif params.Vi > params.Vf:
psi_2 = params.phi_f*np.ones(t_vector_2.size) + params.
psi_scan_rate*t_vector_2

#Concatenate t_vectors and u_matrices into complete data structures for both forward and backward scans.

t_non_dim = np.concatenate((t_vector_1, t_vector_1[-1] + t_vector_2)
, 0)
u_matrix = np.concatenate((u_matrix_1, u_matrix_2), 1)
psi = np.concatenate((psi_1, psi_2), 0)

#Dimensionalize results.
dimensional_sol = dimensionalize_tlm(grid.x, grid.xE, grid.xH,
t_non_dim, u_matrix, psi)
x = dimensional_sol[0]
t = dimensional_sol[1]
P = dimensional_sol[2]
phi = dimensional_sol[3]
n = dimensional_sol[4]
p = dimensional_sol[5]
J_total = dimensional_sol[6]
V_applied = dimensional_sol[7]

#Plot results.
plot_JV(V_applied, J_total, ’dimensional’)
scan1_size = psi_1.size
scan2_size = psi_2.size
return t_non_dim, u_matrix, psi, x, t, P, phi, n, p , J_total,
V_applied, scan1_size, scan2_size




SyntaxError: invalid character '’' (U+2019) (3059648345.py, line 29)